In [1]:
# %%capture
# Install the required libraries
%pip install --upgrade pip
%pip install uv
!uv pip install --system --no-progress --link-mode=symlink --index=https://download.pytorch.org/whl/cu124 accelerate unsloth vllm sentencepiece protobuf datasets huggingface_hub kagglehub[hf-datasets] hf_transfer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 106.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Using Python 3.11.11 environment at: /usr
Resolved 178 packages in 2.72s
 Downloaded tokenizers
 Downloaded torchaudio
 Downloaded xgrammar
 Downloaded uvloop
 Downloaded mistral-common
 Downloaded torchvision
 Downloaded nvidia-nvjitlink-cu12
 Downloaded llguidance
 Downloaded pycountry
 Downloaded llvmlite
 Downloaded hf-xet
 Downloaded numba
 Downloaded nvidia-curand-cu12
 Downloaded bitsandbytes
 Downloaded xformers
 Downloaded nvidia-cusolver-cu12
 Downloaded vllm
 Downloaded nvidia-cusparselt-cu12
 Downloaded triton
 Downloaded nvidia-cusparse-cu1

In [2]:
import re


def normalize_text(text):
    text = re.sub(r"\([\s\u09E6-\u09EF]+\)", " ", text)
    text = re.sub(r"[\u09E6-\u09EF]+", " ", text)
    text = re.sub(r"[\u00A0\u200B\u200C\u200D\u2060\u3000]+", " ", text)
    text = re.sub(r"[ \t\r\f\v]+", " ", text)
    return text.strip()

# Give user an option to either provide a class name or two lines to create a poem
def get_user_input():
    print("Choose an option:")
    print("1. Provide a class name")
    print("2. Provide two lines to create a poem")
    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == "1":
        class_text = input("Enter the class name: ").strip()
        return "class", class_text
    elif choice == "2":
        line1 = input("Enter the first line: ").strip()
        line2 = input("Enter the second line: ").strip()
        return "lines", (line1, line2)
    else:
        print("Invalid choice. Please try again.")
        return get_user_input()
    

# Get user input
user_choice, user_input = get_user_input() 

# Prepare the input for the model
if user_choice == "class":
    class_text = normalize_text(user_input)
    prompt = {
        "Instructions": f"একটি বাংলা কবিতা লেখো যার বিষয় হলো {class_text}। কবিতায় নতুন লাইনের জন্য '\n' এবং নতুন স্তবকের জন্য '\n\n\n' ব্যবহার করো।",
        "Input": class_text,
    }

elif user_choice == "lines":
    line1, line2 = user_input
    line1 = normalize_text(line1)
    line2 = normalize_text(line2)
    poem_text = f"{line1}\n{line2}"
    prompt ={
        "Instructions": f"নিচের দুটি লাইন ব্যবহার করে একটি সম্পূর্ণ বাংলা কবিতা লেখো। কবিতায় নতুন লাইনের জন্য '\n' এবং নতুন স্তবকের জন্য '\n\n\n' ব্যবহার করো।\n{poem_text}",
        "Input": f"{line1}\n{line2}",
    }


Choose an option:
1. Provide a class name
2. Provide two lines to create a poem


Enter your choice (1 or 2):  1
Enter the class name:  চিন্তামূলক


In [7]:
# Enum of models
import enum
class Models(enum.Enum):
    GEMMA_3_4B_CHAT_POEM = "Ankita-Porel/gemma3-4b-bn-chat-poem-ft"
    GEMMA_3_4B_WIKI_POEM = "Venkat423/Gemma-3-4b-finetuned-final"
    SARVAM_1_CHAT_POEM = "Ankita-Porel/sarvam1-bn-chat-poem-ft"
    SARVAM_1_WIKI_POEM = "Ankita-Porel/sarvam1-wiki-poem-bn"

    @classmethod
    def list_model_names(cls):
        return [model.value for model in cls]

    @classmethod
    def list_models(cls):
        return [model.name for model in cls]


In [8]:
# Ask the user to select a model
print("Available models:")
for i, model in enumerate(Models.list_models(), start=1):
    print(f"{i}. {model}")
model_choice = int(input("Select a model (1-4): ")) - 1
if model_choice < 0 or model_choice >= len(Models.list_models()):
    print("Invalid choice. Please try again.")  

model_name = Models.list_model_names()[model_choice]

Available models:
1. GEMMA_3_4B_CHAT_POEM
2. GEMMA_3_4B_WIKI_POEM
3. SARVAM_1_CHAT_POEM
4. SARVAM_1_WIKI_POEM


Select a model (1-4):  1


In [9]:
from unsloth import FastModel


model, tokenizer = FastModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_8bit = False,
    full_finetuning = False,
)

model = FastModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-07 13:34:40.988615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746624881.489544      55 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746624881.646127      55 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-07 13:35:08 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-07 13:35:08 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.1. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.35G [00:00<?, ?B/s]

In [32]:
import os
import torch

input_text = prompt["Instructions"] + " \nকবিতা: \n\n"
input_text = normalize_text(input_text)
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=1024,
).to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        pad_token_id=tokenizer.pad_token_id,
        max_new_tokens=256,
        # do_sample=True,
        # top_k=10,
        # top_p=0.95,
        early_stopping=True,
        # num_return_sequences=1,
    )

completed_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
generated_text = completed_text.removeprefix(input_text)

print("\nInput:")
print(input_text)

print("\nGenerated Poem:")
print(generated_text)

# Save the generated poem to a text file
output_file = "generated_poem.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(generated_text)
print(f"\nPoem saved to {output_file}")


Input:
একটি বাংলা কবিতা লেখো যার বিষয় হলো চিন্তামূলক। কবিতায় নতুন লাইনের জন্য '
' এবং নতুন স্তবকের জন্য '


' ব্যবহার করো। 
কবিতা:

Generated Poem:



এই তো আজ দু'ফুটও কম মানুষ
কেমন দেখে নিতে বলে তিনটি
ল্যাণ্ডমর্মরের গাছ তিনটি
অন্ধকারে करें দরিয়া ভিজা
অনাদৃতের অরণ্য থেকে
অন্ধকার দেখে তোমার সজীব প্রাণ,
চল কীভাবে চলবে মোরে?


১ ปี ধরে তোমার মধ্যে
অনেক স্পষ্ট শব্দ ছিল
কোনো শব্দই দেখলাম না আর,
কেবল শব্দ ছিল অদৃশ্য
শৃংখলাহীন, শব্দহীন, শব্দহীন,
এ কোন শব্দ আমার প্রয়োজন
যদি কোনো না করে, তবু আমার প্রয়োজন।সেদিন ছিল দুপুর। একটা দুপুর
সরে গেছে অনেক আগেই।
কিন্তু এখন তো অকাল দুপুর নয়
হঠাৎ অকাল দুপুর নয়
সুপারফাল্গুনের সূর্য অনেক
সূর্যের চেয়ে লাল আরও
তিলকে এক বিন্দু কোমলা ছিল, কোমলা ছিল।কপাল-লূপণের সেই দুপুর
যে তোমাকে সুন্দর করে
মধু-পণ্ডিত বানিয়েছিল
সেই দুপুর তোমার নয়।
যাকে সে তোমার সুন্দর বলে
সে ছিল অশ্বত্থের নয়, ছিল
জলের নয়, ছিল অশ্বত্থের নয়।তোমার মুখের রঙ সজল吗
আমি জানি পরিবর্তন হল অনেককাল।
ধ

Poem saved to generated_poem.txt
